In [5]:
# import libaries 
import cv2
import numpy as np
import os
# from google.colab.patches import cv2_imshow

# read the video using VideoCapture funtion
cap = cv2.VideoCapture(os.getcwd() + "/data/clip2_7s.mp4")

try:
    while cap.isOpened():
        ret, frame = cap.read()
        cv2.imshow("inter", frame) 
        #   cv2_imshow(frame) # for google colab only

        # esc key = 27
        if cv2.waitKey(40) == 27:
            break

    cv2.destroyAllWindows()
    cap.release

except:
    cv2.destroyAllWindows()
    cap.release


In [3]:
os.getcwd()

'd:\\User\\Coding_Projects\\Python Projects\\Motion-emotion-detection'

In [1]:
# Setting GPU-enabled Tensorflow
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      # tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
      # tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
      tf.config.experimental.set_memory_growth(gpu, True)
      print("sucessful")
  except RuntimeError as e:
    print(e)

sucessful


In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [1]:
# disable gpu
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [2]:
import cv2
from deepface import DeepFace
import numpy as np
# import tensorflow as tf
# # face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
# print(len(tf.config.list_physical_devices('GPU'))>0)

def analyse_face():
    # imagepath = os.getcwd() + "/data/happy_face_woman.png"
    imagepath = os.getcwd() + "/data/face.png"
    # imagepath = os.getcwd() + "/data/class.png"
    image = cv2.imread(imagepath)
    # face_analysis = DeepFace.analyze(image)
    face_analysis = DeepFace.analyze(image, actions=['emotion'], enforce_detection=False)
    print(face_analysis)

analyse_face()

1/1 [==============================] - 4s 4s/step
{'emotion': {'angry': 27.076569199562073, 'disgust': 0.00011799746744145523, 'fear': 59.964632987976074, 'happy': 12.385643273591995, 'sad': 0.5619456525892019, 'surprise': 0.0007821342478564475, 'neutral': 0.010309615026926622}, 'dominant_emotion': 'fear', 'region': {'x': 0, 'y': 0, 'w': 87, 'h': 92}}


# image face + emotion detection

In [4]:
import cv2
from deepface import DeepFace
import numpy as np
import os

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# read the video using VideoCapture funtion
imagepath = os.getcwd() + "/data/class.png"
image = cv2.imread(imagepath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5)
for (x,y,w,h) in faces:
    temp = (x,y,w,h)
    print(temp)
    print(x, y, w, h)
    image = cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
    # print(frame)
    color = (255,0,0)
    # label = "emotion"
    # image=cv2.putText(image, label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    # label = "{}: {:.2f}%".format(label, max(incorrectMask, mask, withoutMask) * 100)
    # frame=cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    try:
        analyze = DeepFace.analyze(image[y:y+h, x:x+w], actions=['emotion'], enforce_detection=False)
        # if I set the enforce_detection as False, then though MTCNN is not detecting any face in the picture, the library will consider the whole input image as a face and compute its embedding
        # analyze = DeepFace.analyze(image, actions=['emotion'], enforce_detection=False)
        print(analyze)
        label = analyze['dominant_emotion']
        image=cv2.putText(image, label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    except Exception as e:
        print(e)
    # frame[y1:y2, x1:x2]

# cv2.imshow("image", image[temp[1]:temp[1]+temp[3], temp[0]:temp[0]+temp[2]]) 
cv2.imshow("image", image) 
cv2.waitKey(0)
cv2.destroyAllWindows()

(1190, 201, 58, 58)
1190 201 58 58
1/1 [==============================] - 0s 40ms/step
{'emotion': {'angry': 1.3205610858572825, 'disgust': 0.00010861906661095263, 'fear': 0.6596329011330486, 'happy': 0.10968569111057414, 'sad': 3.4279894444929893, 'surprise': 0.008387134857292531, 'neutral': 94.47363549156611}, 'dominant_emotion': 'neutral', 'region': {'x': 2, 'y': 3, 'w': 52, 'h': 52}}
(258, 207, 63, 63)
258 207 63 63
1/1 [==============================] - 0s 39ms/step
{'emotion': {'angry': 17.881336383899235, 'disgust': 0.02310058763570861, 'fear': 10.337976982472956, 'happy': 0.13597975131376094, 'sad': 57.24153263816432, 'surprise': 0.004338572657009261, 'neutral': 14.37573667620043}, 'dominant_emotion': 'sad', 'region': {'x': 0, 'y': 0, 'w': 63, 'h': 63}}
(700, 200, 74, 74)
700 200 74 74
1/1 [==============================] - 0s 48ms/step
{'emotion': {'angry': 4.505497485726922, 'disgust': 0.00010644097935775428, 'fear': 85.11115819878329, 'happy': 8.703607839560975, 'sad': 1.666

In [58]:
analyze['emotion']

{'angry': 4.505497485726922,
 'disgust': 0.00010644097935775428,
 'fear': 85.11115819878329,
 'happy': 8.703607839560975,
 'sad': 1.666227082385247,
 'surprise': 0.0012948668744324054,
 'neutral': 0.012110988603768191}

In [11]:
max(analyze['emotion'], key=analyze['emotion'].get)

'fear'

In [24]:
my_keys = sorted(analyze['emotion'], key=analyze['emotion'].get, reverse=True)[:]
print(my_keys)
print(type(my_keys))

['fear', 'happy', 'angry', 'sad', 'neutral', 'surprise', 'disgust']
<class 'list'>


In [63]:
state_of_mind_list = ['Confusion', 'Satisfaction/Delighted', 'Disappointment/Dissatisfaction', 'Frustrated']
print(state_of_mind_list)
state_of_mind_list.remove('Satisfaction/Delighted')
state_of_mind_list.insert(0, 'Satisfaction/Delighted')
print(state_of_mind_list)


['Confusion', 'Satisfaction/Delighted', 'Disappointment/Dissatisfaction', 'Frustrated']
['Satisfaction/Delighted', 'Confusion', 'Disappointment/Dissatisfaction', 'Frustrated']


In [66]:
confusion_list = ['neutral', 'supprise'] # duplicated combinatorial emotions for 'sad' with 'Disappointment/dissatisfaction', so 'sad' is removed from 'Confusion'
satisfaction_delighterd_list = ['happy', 'neutral']
disappointment_disstisfaction_list = ['neutral', 'sad']
frustrated_list = ['sad', 'angry', 'neutral']
state_of_mind_dict = {'Confusion': confusion_list, 
'Satisfaction/Delighted': satisfaction_delighterd_list, 
'Disappointment/Dissatisfaction': disappointment_disstisfaction_list,
'Frustrated': frustrated_list}
state_of_mind_count = {'Confusion': [0, 0], 
'Satisfaction/Delighted': [0, 0], 
'Disappointment/Dissatisfaction': [0, 0, 0],
'Frustrated': [0, 0, 0]}
state_of_mind_list = ['Confusion', 'Satisfaction/Delighted', 'Disappointment/Dissatisfaction', 'Frustrated']

# pass in a dictionary of confidence score for each emotion
def get_state_of_mind(emotions_dict):
    count = 0
    # get the 3 emotions with highest confidence score
    sorted_emotions = sorted(emotions_dict, key=emotions_dict.get, reverse=True)[:]
    for emotion in sorted_emotions:
        print("emotion: ", emotion)
        # for state in state_of_mind_dict:
        for state in state_of_mind_list:
            print("state: ", state)
            # if the emotion belongs to a certain state of mind
            if emotion in state_of_mind_dict[state]: 
                if count == 0:
                    state_of_mind_list.remove(state)
                    state_of_mind_list.insert(0, state)
                    count = 1
                # get the index of the emotion in the dictionary
                idx = state_of_mind_dict[state].index(emotion)
                print("idx", idx)
                # set the count to 1
                state_of_mind_count[state][idx] = 1
                print(state_of_mind_count)
            
            # check if all emotions of a certain state of mind were matched
            if sum(state_of_mind_count[state]) == len(state_of_mind_count[state]):
                print("matched state of mind!")
                return

get_state_of_mind(analyze['emotion'])

emotion:  fear
state:  Confusion
state:  Satisfaction/Delighted
state:  Disappointment/Dissatisfaction
state:  Frustrated
emotion:  happy
state:  Confusion
state:  Satisfaction/Delighted
idx 0
{'Confusion': [0, 0], 'Satisfaction/Delighted': [1, 0], 'Disappointment/Dissatisfaction': [0, 0, 0], 'Frustrated': [0, 0, 0]}
state:  Disappointment/Dissatisfaction
state:  Frustrated
emotion:  angry
state:  Satisfaction/Delighted
state:  Confusion
state:  Disappointment/Dissatisfaction
state:  Frustrated
idx 1
{'Confusion': [0, 0], 'Satisfaction/Delighted': [1, 0], 'Disappointment/Dissatisfaction': [0, 0, 0], 'Frustrated': [0, 1, 0]}
emotion:  sad
state:  Satisfaction/Delighted
state:  Confusion
state:  Disappointment/Dissatisfaction
idx 1
{'Confusion': [0, 0], 'Satisfaction/Delighted': [1, 0], 'Disappointment/Dissatisfaction': [0, 1, 0], 'Frustrated': [0, 1, 0]}
state:  Frustrated
idx 0
{'Confusion': [0, 0], 'Satisfaction/Delighted': [1, 0], 'Disappointment/Dissatisfaction': [0, 1, 0], 'Frustra

In [60]:
state_of_mind_count

{'Confusion': [1, 1],
 'Satisfaction/Delighted': [1, 1],
 'Disappointment/Dissatisfaction': [1, 0, 0],
 'Frustrated': [1, 1, 1]}

In [50]:
state_of_mind_count['Disappointment/Dissatisfaction'][1] = 999

In [38]:
sorted_emotions = ['fear', 'happy', 'angry', 'sad', 'neutral', 'surprise', 'disgust']

In [56]:
# state_of_mind_list = ['Confusion', 'Satisfaction/Delighted', 'Disappointment/Dissatisfaction', 'Frustrated']
for emotion in sorted_emotions:
    print("emotion: ", emotion)
    for state in state_of_mind_dict:
        print("state: ", state)
        # if the emotion belongs to a certain state of mind
        if emotion in state_of_mind_dict[state]: 
            # get the index of the emotion in the dictionary
            idx = state_of_mind_dict[state].index(emotion)
            print("idx", idx)
            # special condition where for confusion state of mind the emotion is supprise or sad
            if state == 'Confusion' and emotion == 'sad':
                idx = 1
            # set the count to 1
            state_of_mind_count[state][idx] = 1
            print(state_of_mind_count)


emotion:  fear
state:  Confusion
state:  Satisfaction/Delighted
state:  Disappointment/Dissatisfaction
state:  Frustrated
emotion:  happy
state:  Confusion
state:  Satisfaction/Delighted
idx 0
{'Confusion': [0, 0], 'Satisfaction/Delighted': [1, 0], 'Disappointment/Dissatisfaction': [0, 999, 0], 'Frustrated': [0, 1, 0]}
state:  Disappointment/Dissatisfaction
state:  Frustrated
emotion:  angry
state:  Confusion
state:  Satisfaction/Delighted
state:  Disappointment/Dissatisfaction
state:  Frustrated
idx 1
{'Confusion': [0, 0], 'Satisfaction/Delighted': [1, 0], 'Disappointment/Dissatisfaction': [0, 999, 0], 'Frustrated': [0, 1, 0]}
emotion:  sad
state:  Confusion
idx 2
{'Confusion': [0, 1], 'Satisfaction/Delighted': [1, 0], 'Disappointment/Dissatisfaction': [0, 999, 0], 'Frustrated': [0, 1, 0]}
state:  Satisfaction/Delighted
state:  Disappointment/Dissatisfaction
state:  Frustrated
idx 0
{'Confusion': [0, 1], 'Satisfaction/Delighted': [1, 0], 'Disappointment/Dissatisfaction': [0, 999, 0], 

In [46]:
state_of_mind_list = ['Confusion', 'Satisfaction/Delighted', 'Disappointment/Dissatisfaction', 'Frustrated']
for state in state_of_mind_list:
    print(state_of_mind_dict[state])
    for emotion in sorted_emotions:
        print("emotion: ", emotion)
        if emotion in state_of_mind_dict[state]:
            # find the index of the emotion
            idx = state_of_mind_dict[state].index(emotion)
            print("idx", idx)
            if state == 'Confusion' and emotion == 'sad':
                idx = 1
            # set the count to 1
            state_of_mind_count[state][idx] = 1 
            print(state_of_mind_count)

['neutral', 'supprise', 'sad']
emotion:  fear
emotion:  happy
emotion:  angry
emotion:  sad
idx 2


IndexError: list assignment index out of range

In [45]:
state_of_mind_count

{'Confusion': [0, 0],
 'Satisfaction/Delighted': [0, 0],
 'Disappointment/Dissatisfaction': [0, 0, 0],
 'Frustrated': [0, 0, 0]}

# video face detection

In [2]:
import cv2
from deepface import DeepFace
import numpy as np
import os

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# read the video using VideoCapture funtion
video = cv2.VideoCapture(0)
# video = cv2.VideoCapture(os.getcwd() + "/data/clip2_7s.mp4")
# video = cv2.VideoCapture(os.getcwd() + "/data/vid2_part1.mp4")
# video = cv2.VideoCapture(os.getcwd() + "/data/vid1_short.mp4")

try:
    while video.isOpened():
    # read the video into farme
        # video.set(cv2.CAP_PROP_FPS,1) 
        ret, frame = video.read()
        # print(video.get(cv2.CAP_PROP_FPS))
        # convert the frame to grey for CascadeClassifier
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # find coordinates of faces, returns (x,y,w,h)
        # faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5)

        for (x,y,w,h) in faces:
            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            # print(frame)
            color = (255,0,0)
            try:
                analyze = DeepFace.analyze(frame[y:y+h, x:x+w], actions=['emotion'], enforce_detection=False)
                # if I set the enforce_detection as False, then though MTCNN is not detecting any face in the picture, the library will consider the whole input image as a face and compute its embedding
                # analyze = DeepFace.analyze(image, actions=['emotion'], enforce_detection=False)
                # print(analyze)
                label = analyze['dominant_emotion']
                frame=cv2.putText(frame, label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            except Exception as e:
                print(e)
        # frame[y1:y2, x1:x2]

        cv2.imshow("video", frame) 
        #   cv2_imshow(frame) # for google colab only
        cv2.waitKey(5000)
        # if cv2.waitKey(40) == 27:
            # break

    cv2.destroyAllWindows()
    video.release

except Exception as e:
    print(e)
cv2.destroyAllWindows()
video.release

1/1 [==============================] - 0s 37ms/step


KeyboardInterrupt: 

In [1]:
import cv2

# read the video and extract info about it
# cap = cv2.VideoCapture('vid.avi')
cap = cv2.VideoCapture(os.getcwd() + "/data/clip2_7s.mp4")


# get total number of frames and generate a list with each 30 th frame 
totalFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
x = [i for i in range (1, totalFrames) if divmod(i, int(30))[1]==0]


for myFrameNumber in x:
    cap.set(cv2.CAP_PROP_POS_FRAMES,myFrameNumber)
    while True:
        ret, frame = cap.read()
        cv2.imshow("video", frame)

        ch = 0xFF & cv2.waitKey(1000)
        if ch == 27:
            break
cv2.destroyAllWindows()

In [3]:
x

[30, 60, 90, 120, 150]

In [5]:
!pip show tensorflow

Name: tensorflow
Version: 2.9.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\guang\anaconda3\envs\deepfacepy37\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: deepface, retina-face


In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [1]:
!nvidia-smi

Mon Nov 14 22:51:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.83       Driver Version: 451.83       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:5A:00.0 Off |                  N/A |
| N/A   47C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [6]:
!nvidia-smi

Thu Nov 10 01:44:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.83       Driver Version: 451.83       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:5A:00.0 Off |                  N/A |
| N/A   48C    P8    N/A /  N/A |   1432MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  